In [1]:
import numpy as np
import pandas as pd
import dimod
from dwave_qbsolv import QBSolv

# Set random seed for reproducibility
np.random.seed(42)

# Parameters for dataset
total_warehouses = 10
total_suppliers = 5
total_customers = 15

# Generate synthetic warehouses
df_warehouses = pd.DataFrame({
    'Warehouse_ID': range(1, total_warehouses + 1),
    'Capacity': np.random.randint(500, 2000, total_warehouses),
    'Fixed_Cost': np.random.randint(2000, 10000, total_warehouses)
})

# Generate synthetic suppliers
df_suppliers = pd.DataFrame({
    'Supplier_ID': range(1, total_suppliers + 1),
    'Supply_Capacity': np.random.randint(1000, 5000, total_suppliers),
    'Cost_Per_Unit': np.random.randint(5, 20, total_suppliers)
})

# Generate synthetic customer demand
df_customers = pd.DataFrame({
    'Customer_ID': range(1, total_customers + 1),
    'Demand': np.random.randint(100, 500, total_customers)
})

# Generate synthetic transportation costs (Warehouse → Customer)
transport_cost = np.random.randint(10, 50, (total_warehouses, total_customers))

# Binary Decision Variables for QUBO
qubo = {}
penalty = 1000  # Large penalty to enforce constraints

# Variable mapping: x_ij represents sending supply from Warehouse i to Customer j
for i in range(total_warehouses):
    for j in range(total_customers):
        qubo[(f'x_{i}_{j}', f'x_{i}_{j}')] = transport_cost[i, j]  # Minimize cost

# Demand constraint: Ensure customer demand is met
for j in range(total_customers):
    constraint = sum([f'x_{i}_{j}' for i in range(total_warehouses)])
    qubo[(constraint, constraint)] = penalty  # Penalize deviation from demand

# Warehouse capacity constraint: Ensure warehouses do not exceed capacity
for i in range(total_warehouses):
    constraint = sum([f'x_{i}_{j}' for j in range(total_customers)])
    qubo[(constraint, constraint)] = penalty  # Penalize exceeding capacity

# Convert to BQM (Binary Quadratic Model)
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo)

# Solve using QBSolv (simulated annealing approach)
solution = QBSolv().sample(bqm)

# Extract results
qubo_solution = solution.first.sample

# Print optimal warehouse-customer assignments
print("Optimal Supply Assignments:")
for key, value in qubo_solution.items():
    if value == 1:
        print(key, "-> Assigned")


ModuleNotFoundError: No module named 'dimod'